In [ ]:
#Initialize the Gemini Model

Copy

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
)
response = llm.invoke("What is the capital of Canada?")
print(response.content)

The capital of Canada is **Ottawa**.


In [4]:
#Defining Few-Shot Examples
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Albert Einstein?",
        "answer": """Do we need additional questions? Yes.
Steve Jobs died at age 56.
Albert Einstein died at age 76.
Final answer: Albert Einstein."""
    },
    {
        "question": "When was Google's founder born?",
        "answer": """Do we need additional questions? Yes.
Google was founded by Larry Page.
Larry Page was born on March 26, 1973.
Final answer: March 26, 1973."""
    },
]

In [ ]:
#Defining the Example Format

from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

In [7]:
#Creating the FewShotPromptTemplate

from langchain_core.prompts.few_shot import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [9]:
#Executing the Prompt via a Chain

from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

result = chain.invoke({
    "question": "How old was Bill Gates when Microsoft was founded?"
})

print(result)

Do we need additional questions? Yes.
Bill Gates was born on October 28, 1955.
Microsoft was founded on April 4, 1975.
From October 28, 1955, to April 4, 1975, Bill Gates was 19 years old.
Final answer: 19 years old.


In [ ]:
#Semantic Similarity Example Selector (Gemini Embeddings)

from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,          # list of dicts
    embeddings=embeddings,      # embedding function
    vectorstore_cls=Chroma,     # ✅ THIS is required
    k=1,
)

In [ ]:
#Final Chat Prompt Composition
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [ ]:
#Running the Chat Few-Shot Chain
chain = final_prompt | llm

response = chain.invoke({
    "instruction": "Please write meeting minutes",
    "input": "On December 26, the product team reviewed project progress..."
})

print(response.content)

Okay, I can help you write meeting minutes based on that starting point. To make them comprehensive, I'll make some reasonable assumptions about the meeting details. Feel free to adjust any of the assumed information.

---

**MEETING MINUTES**

**Meeting Title:** Product Team Project Progress Review
**Date:** December 26, 2023
**Time:** 10:00 AM - 11:00 AM PST
**Location:** Virtual Meeting (Zoom)

**Attendees:**
*   Sarah Chen (Product Manager - Chair)
*   David Lee (Lead Developer)
*   Maria Rodriguez (UX Designer)
*   Tom Wilson (Marketing Liaison)
*   Emily White (QA Lead - Minute Taker)

**Absentees:** None

---

**1. Review of Project Alpha: "New Feature Rollout"**
*   **Discussion:** Sarah opened the discussion by noting Project Alpha is largely on track for its upcoming beta. David confirmed backend stability and reported that all core functionalities are implemented. Maria presented minor UI feedback received from internal testing, suggesting a few tweaks for improved user expe